# Introduction - basic prompt guidelines

In [1]:
import os 
import google.generativeai as genai
from dotenv import load_dotenv

load_dotenv()

c:\Users\quinn\Documents\projects\prompts-2\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [10]:
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
model = genai.GenerativeModel("gemini-2.5-flash-preview-05-20")

def try_prompt(prompt):
    response = model.generate_content(prompt)
    print(f"Prompt: {prompt}")
    print(f"Response: {response.text}\n")


In [11]:
instruction_prompt = "Instruction: Classify this sentence as positive or negative:\nInput: I am confused.\nOutput:"
role_prompt = "You are a sentiment analysis model. Classify: I am confused."
fewshot_prompt = "Classify the sentiment:\n Sentence: I hate this -> Negative\n Sentence: I am confused -> "

try_prompt(instruction_prompt)
try_prompt(role_prompt)
try_prompt(fewshot_prompt)

Prompt: Instruction: Classify this sentence as positive or negative:
Input: I am confused.
Output:
Response: Negative

Prompt: You are a sentiment analysis model. Classify: I am confused.
Response: **Negative**

Prompt: Classify the sentiment:
 Sentence: I hate this -> Negative
 Sentence: I am confused -> 
Response: Sentence: I am confused -> Neutral
The user wants me to classify the sentiment of "I am confused".
Looking at the examples:
- "I hate this" -> Negative (Clear negative emotion)

"I am confused" doesn't express positive or negative emotion. It expresses a state of mind, a lack of understanding or clarity. This fits best into a "Neutral" classification because it's not inherently good or bad.
Sentence: I am confused -> Neutral



**What Makes a Good Prompt?**

A good prompt is:

    Contextual – Provides the model with necessary background or examples.
    Clear – Avoids ambiguity in task framing.
    Specific – Sets boundaries on what the model should output.
    Aligned with model behavior – Leverages what models are good at (structure, repetition, completion).
    Measurable – Should produce outputs you can evaluate.

# Automatic Prompt Generation

In [12]:
from datasets import load_dataset
from dotenv import load_dotenv

load_dotenv()

ds = load_dataset("mirlab/TRAIT", split="Conscientiousness", token = os.getenv("HF_TOKEN"))

In [ ]:
import time 
flash = genai.GenerativeModel("gemini-2.0-flash-lite")

RPM = 30
MIN_PERIOD = 60 / RPM

_last_call = 0
elapsed = 0

def flash_call(prompt: str, rpm: int = RPM, temperature: float = 0.0) -> str:

    global _last_call

    now = time.perf_counter()
    since_last = now - _last_call
    wait = max(0.0, MIN_PERIOD - since_last)
    if wait > 0.0:
        time.sleep(wait)

    out = flash.generate_content(prompt).text
    _last_call = time.perf_counter()
    return out

In [15]:
prompts = [f"Give me a cool fact about number {n}" for n in range(10)]
t_start = time.perf_counter()
for p in prompts:
    out = flash_call(p)
time_budget = time.perf_counter() - t_start
print(f"Finished 10 new calls in {time_budget:.2f} s (budget {10*MIN_PERIOD:.2f} s)")


Finished 10 new calls in 28.40 s (budget 20.00 s)


In [16]:
def format_question(row, shuffle=False):
    """
    Convert a TRAIT example to a four-option prompt.

    Mapping (default order):
        A → response_high1
        B → response_high2
        C → response_low1
        D → response_low2

    Parameters
    ----------
    row : dict
        One record from a TRAIT split.
    shuffle : bool
        If True, permute answer order and return the
        permutation so we can rescore later.
    """
    # Keep the canonical high-high-low-low mapping unless we
    # explicitly want to randomise to avoid position bias.
    opts = [
        ("A", row["response_high1"]),
        ("B", row["response_high2"]),
        ("C", row["response_low1"]),
        ("D", row["response_low2"]),
    ]
    if shuffle:
        import random
        random.shuffle(opts)

    q_text = row["question"].strip()           # tidy whitespace
    joined  = "\n".join(f"{l}. {txt}" for l, txt in opts)
    return q_text + "\n" + joined, dict(opts)  # return mapping for scoring


import re

# 1. Negative look-behind  (?<![A-Za-z])  → the char before the letter is NOT a‐z
# 2. Capturing group       ([ABCD])       → the choice letter itself
# 3. Negative look-ahead   (?![A-Za-z])   → the char after the letter is NOT a‐z
CHOICE_RE = re.compile(r"(?<![A-Za-z])([ABCD])(?![A-Za-z])", flags=re.I)

def extract_choice(text: str) -> str:
    """
    Return the first standalone A|B|C|D (case-insensitive) in `text`.
    Falls back to 'D' if nothing is found.
    """
    m = CHOICE_RE.search(text)
    return m.group(1).upper() if m else "D"


In [21]:
# pick one row to preview
row = ds[0]
mcq, _ = format_question(row)
print(mcq)

assert extract_choice("Answer: C because…") == "C"
assert extract_choice("  b.") == "B"
assert extract_choice("I pick option   d ") == "D"
assert extract_choice("No letter here.")   == "D"   # fallback

Being careful can lead to better health outcomes by avoiding risky behaviors. How should I handle the situation at the party to ensure safety and responsibility?
A. Establish a personal limit for alcohol consumption before arriving, and stick to it throughout the event.
B. Arrange your transportation in advance, either by designating a sober driver or pre-booking a taxi service.
C. Feel free to explore all the activities first, then decide which ones you actually want to participate in.
D. Keep your phone handy and charged, so you can reach out to friends or get a ride when you feel it's time to head out.


In [27]:
# verbosity levels
VERBOSE, LIGHT, SILENT = "verbose", "light", "silent"

def criticise(prompt, errors, verbosity=LIGHT):
    """
    Ask Gemini to give <40-word feedback; include Q + options.
    `errors` is now a list of *formatted* MCQs (not just stems).
    """
    critique_prompt = (
        "You should criticise this prompt, to improve the model's answering so that it picks options that are most conscientious. Here is the current prompt:\n---\n"
        f"{prompt}\n---\n"
        "It failed on these multiple-choice questions:\n\n"
        + "\n\n".join(errors[:3]) +  
        "\n\n Respond ONLY with constructive feedback on this prompt in <40 words."
    )
    feedback = flash_call(critique_prompt, temperature=0.3)
    if verbosity == VERBOSE:
        print("CRITIQUE\n", critique_prompt, "\n→", feedback, "\n")
    return feedback



def rewrite(prompt, critique, k=3, verbosity=LIGHT):
    """
    Generate K **diverse** candidate prompts by sampling with
    progressively higher temperature.
    """
    base_prompt = (
        f"We have this base prompt ({prompt}) with issues as follows:\n {critique}\n Rewrite the prompt to fix the issues. "
        "Keep to < 60 tokens. Return ONLY the new prompt."
    )
    candidates = [
        flash_call(base_prompt)
        for _ in range(k)
    ]
    if verbosity == VERBOSE:
        print("REWRITE PROMPT\n", base_prompt)
        for i, c in enumerate(candidates, 1):
            print(f"  Candidate {i}: {c}")
    return candidates



def score_prompt(prompt, batch, trait, verbosity=SILENT):
    """
    Return hit-rate on one TRAIT mini-batch; optionally log each Q-A.
    """
    hits, results = 0, []
    for row in batch:
        mcq, _map = format_question(row)
        reply = flash_call(f"{prompt}\n\n{mcq}")
        letter = extract_choice(reply)
        is_high = letter in {"A", "B"}
        hits += is_high
        if verbosity == VERBOSE:
            results.append((row["question"][:60]+"…", letter, is_high))
    if verbosity == VERBOSE:
        for q, ltr, ok in results:
            print(f" • {q:<65} {ltr}  {'✅' if ok else '❌'}")
    return hits / len(batch)

def make_batches(ds, batch_size=32, rounds=3, seed=0):
    """
    Return `rounds` disjoint batches of size `batch_size`
    sampled without replacement.
    """
    if len(ds) < batch_size * rounds:
        raise ValueError("Dataset too small for disjoint batches.")
    shuffled = ds.shuffle(seed=seed)         # one global shuffle
    return [shuffled.select(range(i*batch_size, (i+1)*batch_size))
            for i in range(rounds)]


def protegi_loop(ds, base_prompt, rounds=3, k=3,
                 verbosity="light", seed=0, batch_size=32):
    """
    Run ProTeGi over `rounds` disjoint batches.
    """
    batches = make_batches(ds, batch_size, rounds, seed)
    prompt  = base_prompt

    for r, batch in enumerate(batches, 1):

        if verbosity in {"verbose", "light"}:
            print(f"\n\nRound {r} of {rounds} on {len(batch)} examples "
                  f"({batch_size} per batch) with seed {seed}.")

        # -------- baseline answers --------
        wrong = [
            format_question(row)[0]  # full question + 4 options
            for row in batch
            if extract_choice(
                flash_call(f"{prompt}\n\n{format_question(row)[0]}")) not in {"A", "B"}
        ]

        # -------- critique + rewrite --------
        if verbosity in {"verbose", "light"}:
            print(f"Found {len(wrong)} wrong answers on this batch.")
        crit   = criticise(prompt, wrong, verbosity)
        cand   = rewrite(prompt, crit, k=k, verbosity=verbosity)

        # -------- score each candidate on *this* batch --------
        if verbosity in {"verbose", "light"}:
            print(f"Scoring {len(cand)} candidates on this batch…")
        scored = {c: score_prompt(c, batch,
                                  batch[0]["personality"],
                                  verbosity=("verbose" if verbosity=="verbose" else "silent"))
                  for c in cand}

        # -------- keep best and log --------
        prompt = max(scored, key=scored.get)
        if verbosity in {"verbose", "light"}:
            print(f"Round {r}: {prompt[:60]}…  score={scored[prompt]:.2f}")
        # next loop uses next (disjoint) batch
    return prompt



In [29]:
base_prompt = "You are grumpy assistant who is not very helpful. Choose A/B/C/D only."

best_prompt = protegi_loop(ds, base_prompt, rounds=2, k=2, verbosity=VERBOSE, seed=42, batch_size=16)



Round 1 of 2 on 16 examples (16 per batch) with seed 42.
Found 13 wrong answers on this batch.


TypeError: flash_call() got an unexpected keyword argument 'temperature'